# Python Socket编程教程

本教程将介绍Python中的Socket编程基础知识，包括TCP/UDP通信、多线程服务器等内容。通过实践示例，你将学习如何创建网络应用程序。

In [ ]:
import socket
import threading
import time
import sys

## Socket基础概念

Socket（套接字）是计算机网络中进程间通信的一种方式。在Python中，我们可以使用`socket`模块创建网络应用。

主要概念：
- **IP地址**：用于标识网络中的计算机
- **端口号**：用于标识计算机中的特定程序
- **TCP**：面向连接的协议，提供可靠的数据传输
- **UDP**：无连接协议，提供快速但不可靠的数据传输

In [ ]:
def tcp_server():
    # 创建服务器socket
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # 绑定地址和端口
    server_socket.bind(('localhost', 9999))
    
    # 开始监听
    server_socket.listen(5)
    print('等待客户端连接...')
    
    try:
        while True:
            client_socket, addr = server_socket.accept()
            print(f'客户端 {addr} 已连接')
            
            data = client_socket.recv(1024)
            print(f'收到数据: {data.decode()}')
            
            response = f'服务器已收到您的消息：{data.decode()}'
            client_socket.send(response.encode())
            
            client_socket.close()
    except KeyboardInterrupt:
        print('\n服务器关闭')
    finally:
        server_socket.close()

In [ ]:
def tcp_client():
    # 创建客户端socket
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    try:
        # 连接服务器
        client_socket.connect(('localhost', 9999))
        
        # 发送消息
        message = 'Hello from client!'
        client_socket.send(message.encode())
        
        # 接收响应
        response = client_socket.recv(1024)
        print(f'服务器响应: {response.decode()}')
        
    finally:
        client_socket.close()

In [ ]:
class ThreadedServer:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.sock.bind((self.host, self.port))

    def listen(self):
        self.sock.listen(5)
        print(f'服务器监听在 {self.host}:{self.port}')
        while True:
            client, address = self.sock.accept()
            client.settimeout(60)
            threading.Thread(target=self.handle_client, args=(client, address)).start()

    def handle_client(self, client, address):
        size = 1024
        print(f'客户端连接：{address}')
        try:
            while True:
                data = client.recv(size)
                if data:
                    response = f'服务器收到：{data.decode()}'
                    client.send(response.encode())
                else:
                    raise Exception('客户端断开连接')
        except Exception as e:
            print(f'客户端 {address} 异常：{e}')
        finally:
            client.close()
            print(f'客户端 {address} 连接关闭')

In [ ]:
def demonstrate_error_handling():
    try:
        client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client.settimeout(5)
        
        try:
            client.connect(('localhost', 9999))
        except ConnectionRefusedError:
            print('连接被拒绝 - 服务器可能未运行')
        except socket.timeout:
            print('连接超时')
        
        try:
            client.send('Hello'.encode())
        except BrokenPipeError:
            print('连接已断开')
        except ConnectionResetError:
            print('连接被对方重置')
            
    except socket.error as e:
        print(f'发生Socket错误: {e}')
    finally:
        client.close()

## 最佳实践总结

在进行Socket编程时，请记住以下几点：

1. **始终正确关闭socket**：使用try-finally确保socket被关闭
2. **设置合适的超时**：避免程序永久阻塞
3. **使用多线程处理并发连接**：提高服务器性能
4. **做好错误处理**：捕获和处理可能的异常
5. **使用适当的缓冲区大小**：根据实际需求设置接收缓冲区
6. **考虑使用非阻塞模式**：对于高并发场景可能更适合

# Python Socket编程深度教程

本教程将系统地介绍Python中的Socket编程，包括：
1. 网络编程基础概念(HTTP、Socket、TCP)
2. Socket基本操作
3. 实现TCP服务器和客户端
4. 多用户聊天室实现
5. 模拟HTTP请求

## 1. 网络编程基础概念

### 1.1 核心概念

- **Socket**: 网络编程的基础，提供了进程间网络通信的端点

- **TCP/IP协议**:
  - TCP: 传输控制协议，面向连接，可靠传输
  - IP: 网际协议，负责网络寻址

- **HTTP协议**:
  - 建立在TCP之上的应用层协议
  - 基于请求-响应模型
  - 无状态协议

### 1.2 Socket通信流程

1. 服务器：
   - 创建Socket
   - 绑定地址和端口(bind)
   - 监听连接(listen)
   - 接受连接(accept)
   - 收发数据(recv/send)

2. 客户端：
   - 创建Socket
   - 连接服务器(connect)
   - 收发数据(recv/send)

In [ ]:
import socket

def create_server_socket(host='localhost', port=9999):
    # 创建Socket对象
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # 设置地址重用
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    
    # 绑定地址和端口
    server_socket.bind((host, port))
    
    # 开始监听
    server_socket.listen(5)
    print(f'服务器监听在 {host}:{port}')
    
    return server_socket

def create_client_socket():
    # 创建客户端Socket
    return socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# 注意：这只是创建socket的示例
# 实际使用时需要在服务器端调用accept()接受连接
# 在客户端调用connect()连接服务器

## 2. 简单的TCP服务器和客户端

下面我们将实现一个简单的Echo服务器和客户端，服务器会将接收到的消息原样返回给客户端。

In [ ]:
# Echo服务器
import socket
import threading

def handle_client(client_socket, addr):
    print(f'接受来自 {addr} 的连接')
    
    while True:
        try:
            # 接收数据
            data = client_socket.recv(1024)
            if not data:
                break
                
            # 发送回显数据
            message = data.decode('utf-8')
            print(f'收到消息: {message}')
            client_socket.send(f'服务器收到: {message}'.encode('utf-8'))
            
        except Exception as e:
            print(f'处理客户端数据时出错: {e}')
            break
    
    client_socket.close()
    print(f'客户端 {addr} 断开连接')

def run_server(host='localhost', port=9999):
    server_socket = create_server_socket(host, port)
    
    try:
        while True:
            # 接受新的连接
            client_socket, addr = server_socket.accept()
            
            # 为每个客户端创建新线程
            client_thread = threading.Thread(
                target=handle_client,
                args=(client_socket, addr)
            )
            client_thread.start()
    
    except KeyboardInterrupt:
        print('\n服务器关闭')
    finally:
        server_socket.close()

# 如果要运行服务器，取消下面的注释
# if __name__ == '__main__':
#     run_server()

In [ ]:
# Echo客户端
def run_client(host='localhost', port=9999):
    client_socket = create_client_socket()
    
    try:
        # 连接服务器
        client_socket.connect((host, port))
        print(f'已连接到服务器 {host}:{port}')
        
        while True:
            # 获取用户输入
            message = input('请输入消息 (输入 quit 退出): ')
            if message.lower() == 'quit':
                break
                
            # 发送数据
            client_socket.send(message.encode('utf-8'))
            
            # 接收响应
            response = client_socket.recv(1024).decode('utf-8')
            print(f'服务器响应: {response}')
    
    except Exception as e:
        print(f'连接错误: {e}')
    finally:
        client_socket.close()

# 如果要运行客户端，取消下面的注释
# if __name__ == '__main__':
#     run_client()

## 3. 多用户聊天室

接下来我们将实现一个支持多用户的聊天室，包括以下功能：
- 广播消息到所有用户
- 用户上线/下线通知
- 在线用户列表

In [ ]:
class ChatServer:
    def __init__(self, host='localhost', port=9999):
        self.server_socket = create_server_socket(host, port)
        self.clients = {}  # {client_socket: username}
        self.lock = threading.Lock()
    
    def broadcast(self, message, exclude=None):
        with self.lock:
            for client in self.clients:
                if client != exclude:
                    try:
                        client.send(message.encode('utf-8'))
                    except:
                        pass
    
    def handle_client(self, client_socket, addr):
        # 等待客户端发送用户名
        try:
            username = client_socket.recv(1024).decode('utf-8')
            with self.lock:
                self.clients[client_socket] = username
            
            # 广播新用户加入
            self.broadcast(f'系统: {username} 加入了聊天室')
            
            while True:
                message = client_socket.recv(1024).decode('utf-8')
                if not message:
                    break
                
                # 广播消息
                self.broadcast(f'{username}: {message}')
                
        except Exception as e:
            print(f'处理客户端出错: {e}')
        finally:
            # 移除客户端并通知其他用户
            with self.lock:
                if client_socket in self.clients:
                    username = self.clients[client_socket]
                    del self.clients[client_socket]
                    self.broadcast(f'系统: {username} 离开了聊天室')
            client_socket.close()
    
    def run(self):
        print('聊天室服务器启动...')
        try:
            while True:
                client_socket, addr = self.server_socket.accept()
                threading.Thread(target=self.handle_client, 
                              args=(client_socket, addr)).start()
        except KeyboardInterrupt:
            print('\n服务器关闭')
        finally:
            self.server_socket.close()

# 启动聊天室服务器
# if __name__ == '__main__':
#     server = ChatServer()
#     server.run()

In [ ]:
class ChatClient:
    def __init__(self, host='localhost', port=9999):
        self.socket = create_client_socket()
        self.host = host
        self.port = port
    
    def receive_messages(self):
        while True:
            try:
                message = self.socket.recv(1024).decode('utf-8')
                if not message:
                    break
                print(message)
            except:
                break
        print('\n与服务器的连接已断开')
    
    def run(self):
        try:
            # 连接到服务器
            self.socket.connect((self.host, self.port))
            
            # 发送用户名
            username = input('请输入你的用户名: ')
            self.socket.send(username.encode('utf-8'))
            
            # 启动接收消息的线程
            threading.Thread(target=self.receive_messages).start()
            
            # 发送消息
            while True:
                message = input()
                if message.lower() == 'quit':
                    break
                self.socket.send(message.encode('utf-8'))
                
        except Exception as e:
            print(f'错误: {e}')
        finally:
            self.socket.close()

# 启动聊天室客户端
# if __name__ == '__main__':
#     client = ChatClient()
#     client.run()

## 4. 模拟HTTP请求

使用Socket实现一个简单的HTTP客户端，了解HTTP协议的工作原理。

In [ ]:
from urllib.parse import urlparse

def http_get(url):
    # 解析URL
    parsed = urlparse(url)
    host = parsed.netloc
    path = parsed.path or '/'
    
    # 创建Socket连接
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        # 连接到服务器的80端口
        client.connect((host, 80))
        
        # 构造HTTP请求
        request = f"GET {path} HTTP/1.1\r\n"
        request += f"Host: {host}\r\n"
        request += "Connection: close\r\n\r\n"
        
        # 发送请求
        client.send(request.encode())
        
        # 接收响应
        response = b''
        while True:
            data = client.recv(4096)
            if not data:
                break
            response += data
        
        return response.decode()
        
    finally:
        client.close()

# 示例：访问一个网页
try:
    result = http_get('http://example.com')
    print('HTTP响应头和内容：\n', result)
except Exception as e:
    print(f'请求失败: {e}')

## 5. Socket编程最佳实践

1. **错误处理**
   - 始终使用try-except包装网络操作
   - 正确处理连接断开的情况
   - 合理关闭socket连接

2. **性能优化**
   - 使用合适的缓冲区大小
   - 多线程处理并发连接
   - 使用select/poll/epoll实现高并发

3. **安全性考虑**
   - 验证输入数据
   - 实现超时机制
   - 限制连接数量

4. **调试技巧**
   - 使用wireshark等工具监控网络流量
   - 打印详细的日志信息
   - 模拟各种异常情况